# Data Prep

### Data Fetch From Trino


In [ ]:
"""
This script establishes a connection to a Presto database via the Trino Python client, 
executes a SQL query to fetch data, and then structures the fetched data into a pandas DataFrame.
The DataFrame is Clusters with appropriate column names, making the data ready for analysis.
"""

import trino
import pandas as pd
import datetime

# Connect to Trino
conn = trino.dbapi.connect(
    host="presto-gateway.corp.mongodb.com",
    port=443,
    user="jiawei.zhou@mongodb.com",
    catalog="awsdatacatalog",
    http_scheme="https",
    auth=trino.auth.BasicAuthentication(
        "jiawei.zhou@mongodb.com", "Youarethebest@1016"
    ),
)

# Execute query
cur = conn.cursor()

In [ ]:
cur.execute(
    f"""
select
	*
from
	awsdatacatalog.product_analytics.pnl_products_adoption_qualification_activation
"""
)

# Extract column names
columns = [desc[0] for desc in cur.description]

# Fetch rows
rows = cur.fetchall()

# Create DataFrame
df_raw = pd.DataFrame(rows, columns=columns)

# Convert 'ds' column to pandas datetime
df_raw["ds"] = pd.to_datetime(df_raw["ds"])

# Now perform the filtering with no type mismatch
df_raw = df_raw[
    (df_raw["ds"] >= pd.Timestamp("2023-02-01"))
    & (df_raw["ds"] <= pd.Timestamp("2024-02-01"))
]

In [ ]:
# Set the maximum number of displayed rows
pd.set_option("display.max_rows", 100)

# Set the maximum number of displayed columns
pd.set_option("display.max_columns", 100)

# Data Profiling

In [ ]:
# # Base product name
# product = "text_search"

# Dynamically generate column names based on the product
product_ds = f"{product}_ds"
product_ind = f"{product}_ind"

# Print the results to verify
print(product_ds)
print(product_ind)

In [ ]:
# Create a copy of the original DataFrame
data = df_raw.copy()

# Display rows
data.head()

In [ ]:
data["ds"].value_counts().sort_index()

In [ ]:
import pandas as pd

# Set pandas option to display float format without scientific notation
pd.set_option("display.float_format", "{:.2f}".format)

# Use the describe function and print the result
data.describe()

### Null & 0 Values

In [ ]:
import pandas as pd

# Calculate the total number of clusters (rows in your DataFrame)
total_clusters = data.shape[0]

# Calculate the null values count for each column
null_values = data.isnull().sum()

# Calculate the percentage of null values for each column
null_values_percentage = (null_values / total_clusters) * 100

# Calculate the 0 values count for each column
zero_values = (data == 0).sum()

# Calculate the percentage of 0 values for each column
zero_values_percentage = (zero_values / total_clusters) * 100

# Create a DataFrame to display the null values count and percentage
null_values_df = pd.DataFrame(
    {"Null Count": null_values, "Null Percentage": null_values_percentage}
)

# Create a DataFrame to display the 0 values count and percentage
zero_values_df = pd.DataFrame(
    {"Zero Count": zero_values, "Zero Percentage": zero_values_percentage}
)

# Merge the two DataFrames for a comprehensive view
values_df = pd.concat([null_values_df, zero_values_df], axis=1)

# Format the percentage values to string with a '%' sign for better readability
values_df = values_df.applymap(lambda x: f"{x:.2f}%" if isinstance(x, float) else x)

# Display rows where the Null Count or Zero Count is greater than 0
filtered_values_df = values_df[
    (values_df["Null Count"] > 0) | (values_df["Zero Count"] > 0)
]

# If you want to see the result
filtered_values_df

In [ ]:
import pandas as pd

# Calculate the total number of clusters (rows in your DataFrame)
total_clusters = data.shape[0]

# Calculate the percentage of null values for each column
null_values_percentage = (data.isnull().sum() / total_clusters) * 100

# Identify columns to exclude from dropping (i.e., keep these columns regardless of their null percentage)
columns_to_keep = [product_ds, product_ind]

# Find columns with >= 40% null values, excluding the specified columns
columns_to_drop = [
    col
    for col in null_values_percentage.index
    if null_values_percentage[col] >= 40 and col not in columns_to_keep
]

# Drop these columns from the DataFrame
data = data.drop(columns=columns_to_drop)

### Variables Distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Convert 'ds' to datetime format
data["ds"] = pd.to_datetime(data["ds"])

# Filtering the data for 'ds = 2023-02-01'
filtered_data = data[data["ds"] == "2023-02-01"]

# Update the lists of features as per your specification
categorical_features = [
    "org_plan_type",
    "is_cross_region",
    "instance_size",
    "topology",
    "cluster_mdb_major_version",
    "org_country",
    "email_segment_clean",
    "marketing_channel_group",
    "account_segment",
    "self_serve_or_sales_sold",
]

numerical_features = [
    "cluster_age_month",
    "disk_size_gb",
    "org_latest_mrr",
    "reads_per_second_monthly_daily_avg",
    "writes_per_second_monthly_daily_avg",
    "connections_monthly_daily_avg",
    "opcounter_cmd_monthly_daily_avg",
    "db_data_size_monthly_daily_avg",
    "cluster_mrr_total",
]

# Calculate the number of plots needed
num_plots = len(categorical_features) + len(numerical_features)
cols = 2
rows = num_plots // cols + (num_plots % cols > 0)

plt.figure(
    figsize=(16, max(5 * rows, 20))
)  # Adjusting the figure size based on the number of plots

# Plotting histograms for the new set of numerical variables
for i, var in enumerate(numerical_features, 1):
    plt.subplot(rows, cols, i)
    sns.histplot(filtered_data[var], kde=True, bins=30)
    plt.title(f"Distribution of {var} for 2023-02-01")

# Plotting bar plots for the new set of categorical variables
for j, var in enumerate(categorical_features, len(numerical_features) + 1):
    plt.subplot(rows, cols, j)
    order = filtered_data[var].value_counts().index
    sns.countplot(y=var, data=filtered_data, order=order)
    plt.title(f"Distribution of {var} for 2023-02-01")

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Calculate the distribution of the product_ind target variable over the 'ds' date
target_distribution_over_ds = (
    data.groupby("ds")[product_ind].value_counts().unstack(fill_value=0)
)

# Convert the index (which is 'ds') to datetime objects first
target_distribution_over_ds.index = pd.to_datetime(target_distribution_over_ds.index)

# Now, you can safely extract the date part only to ignore the time part
target_distribution_over_ds.index = target_distribution_over_ds.index.date

# Use the parameter in the plot
target_distribution_over_ds.plot(kind="bar", stacked=True, figsize=(10, 5))

# Integrate the product_title into the title and legend labels
plt.title(f"Distribution of {product} Indicator Over Date")
plt.xlabel("Date (ds)")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.legend(
    labels=[f"w/o {product}", f"w/ {product}"],
    loc="upper left",
)
plt.tight_layout()
plt.grid(False)
plt.show()

# Matching Process

### Assigning Treatment & Control

In [ ]:
import pandas as pd

# Create a copy of the original DataFrame
data = df_raw.copy()

data["ds"] = pd.to_datetime(data["ds"])
data[product_ds] = pd.to_datetime(data[product_ds])

# Determine the earliest product_ds date for each cluster
earliest_product = data.groupby("cluster_id")[product_ds].min().reset_index()


def assign_group(row):
    # Convert string dates to datetime objects for comparison
    start_date = pd.to_datetime("2023-02-01")
    end_date = pd.to_datetime("2023-12-31")

    if pd.isnull(row[product_ds]):
        return "control group"
    elif row[product_ds] < start_date or row[product_ds] > end_date:
        return "not eligible"
    elif start_date <= row[product_ds] <= end_date:
        return "treatment group"
    else:
        return "control group"


earliest_product["group_assignment"] = earliest_product.apply(assign_group, axis=1)


# Merge the group assignment back to the original dataset
data = pd.merge(
    data,
    earliest_product[["cluster_id", "group_assignment"]],
    on="cluster_id",
    how="left",
)

In [ ]:
# Ensure product_ds is in datetime format
data[product_ds] = pd.to_datetime(data[product_ds], errors="coerce")

# 1. Count of all unique clusters
total_unique_clusters = data["cluster_id"].nunique()

# 2. Count of unique clusters with product_ind equal to 1
unique_clusters_product_ind_1 = data[data[product_ind] == 1]["cluster_id"].nunique()

# 3. Count of unique clusters with product_ds >= '2023-02-01' and <= '2023-12-31'
unique_clusters_product_ds = data[
    (data[product_ds] >= "2023-02-01") & (data[product_ds] <= "2023-12-31")
]["cluster_id"].nunique()

# 4. Count of unique clusters in each 'group_assignment'
unique_clusters_per_group_assignment = data.groupby("group_assignment")[
    "cluster_id"
].nunique()

# Display the results with dynamic column names
print(f"Total number of unique clusters: {total_unique_clusters}")
print(f"Unique clusters with '{product_ind}' = 1: {unique_clusters_product_ind_1}")
print(
    f"Unique clusters with '{product_ds}' >= '2023-02-01' and < '2023-12-31': {unique_clusters_product_ds}"
)
print("Unique clusters in each 'group_assignment':")
for group_assignment, count in unique_clusters_per_group_assignment.items():
    print(f"  - {group_assignment}: {count}")

In [ ]:
import pandas as pd
import numpy as np

# Convert 'ds' and product_ds to datetime format in one step
data[["ds", product_ds]] = data[["ds", product_ds]].apply(pd.to_datetime)

# Define the filtering conditions in advance to improve readability
eligible = data["group_assignment"] != "not eligible"
date_range = (data["ds"] >= "2023-02-01") & (data["ds"] <= "2023-12-31")
not_deleted = data["is_deleted"] != True  # Add not deleted filter
mrr_greater_than_zero = data["cluster_mrr_total"] > 0  # Add MRR > 0 filter

# Apply all filtering conditions
data = data[eligible & date_range & not_deleted & mrr_greater_than_zero]

# Use a single line to create 'same_month_indicator' by comparing years and months directly
data["same_month_indicator"] = (
    (data["ds"].dt.to_period("M") == data[product_ds].dt.to_period("M"))
).astype(int)

# Convert 'ds' to a 'month_year' period format for monthly analysis directly
data["month_year"] = data["ds"].dt.to_period("M")

# Set 'product_month_indicator' using period comparison for simplicity
data["product_month_indicator"] = (
    data[product_ds].notnull()
    & (data[product_ds].dt.to_period("M") <= data["month_year"])
).astype(int)

In [ ]:
# Group by 'product_ind' and count unique 'cluster_id's
unique_clusters_product_ind = data.groupby(product_ind)["cluster_id"].nunique()

# Use the dynamically generated 'product_ind' in the print statement
print(f"Unique clusters by {product_ind}:\n", unique_clusters_product_ind)

# Group by 'same_month_indicator' and count unique 'cluster_id's
unique_clusters_same_month_indicator = data.groupby("same_month_indicator")[
    "cluster_id"
].nunique()
print(
    "\nUnique clusters by same_month_indicator:\n", unique_clusters_same_month_indicator
)

# Group by 'group_assignment' and count unique 'cluster_id's
unique_clusters_group_assignment = data.groupby("group_assignment")[
    "cluster_id"
].nunique()
print("\nUnique clusters by group_assignment:\n", unique_clusters_group_assignment)

### Check Missing Values

In [ ]:
import pandas as pd

# Convert 'ds' column to datetime format
data["ds"] = pd.to_datetime(data["ds"])

# Filter data for a specific date
filtered_data = data[data["ds"] == "2023-02-01"]

# Count missing values by 'group_assignment'
missing_values = filtered_data.isnull().groupby(filtered_data["group_assignment"]).sum()

# Add a row to show total missing values for each column
missing_values.loc["Total Missing"] = missing_values.sum()

# Calculate and print the total number of unique clusters per 'group_assignment'
total_clusters = filtered_data.groupby("group_assignment")["cluster_id"].nunique()

# Print the missing values table
missing_values

In [ ]:
# Define features and target
features = [
    "cluster_age_month",
    "org_plan_type",
    "is_cross_region",
    "instance_size",
    "topology",
    "cluster_mdb_major_version",
    "disk_size_gb",
    "org_country",
    "org_latest_mrr",
    "email_segment_clean",
    "marketing_channel_group",
    "self_serve_or_sales_sold",
    "reads_per_second_monthly_daily_avg",
]

# Ensure 'ds' column is in datetime format (if not already done)
data["ds"] = pd.to_datetime(data["ds"])

# Filter the DataFrame for rows where 'ds' equals '2023-02-01'
filtered_data = data[data["ds"] == "2023-02-01"]

# Calculate the percentage of rows with any missing value, grouped by 'group_assignment'
# Only considering the filtered data for '2023-02-01'
percentage_missing = (
    filtered_data[features]
    .isnull()
    .any(axis=1)
    .groupby(filtered_data["group_assignment"])
    .mean()
    * 100
)

# Format the output with a '%' sign
formatted_percentage_missing = percentage_missing.apply(lambda x: f"{x:.2f}%")

# Display the formatted percentages
formatted_percentage_missing

### Logit Regression & Feature Importance

In [ ]:
# 1. try without smote
# 2. xgboost no need for the standardization
# 3. shapley value

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import (
    roc_auc_score,
    precision_recall_curve,
    auc,
    cohen_kappa_score,
    matthews_corrcoef,
    log_loss,
)
from sklearn.pipeline import make_pipeline

# Filter dataset for ds='2023-12-01'
data_filtered = data[data["ds"] == "2023-12-01"]

X = data_filtered[features]
y = data_filtered["product_month_indicator"]

# Drop rows with any null values in the features
X = X.dropna()
y = y[X.index]

# Apply Label Encoding to categorical features
le = LabelEncoder()
for column in X.select_dtypes(include=["object"]).columns:
    X[column] = le.fit_transform(X[column])

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Address imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train Logistic Regression classifier within a pipeline that includes scaling
model = make_pipeline(StandardScaler(), LogisticRegression(random_state=42))
model.fit(X_train_smote, y_train_smote)

# Predict and evaluate the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[
    :, 1
]  # Probability estimates for the positive class

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Calculate and print additional evaluation metrics
roc_auc = roc_auc_score(y_test, y_pred_proba)
precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall, precision)
cohen_kappa = cohen_kappa_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
logloss = log_loss(y_test, y_pred_proba)

print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"Precision-Recall AUC: {pr_auc:.4f}")
print(f"Cohen’s Kappa: {cohen_kappa:.4f}")
print(f"Matthews Correlation Coefficient: {mcc:.4f}")
print(f"Log Loss: {logloss:.4f}")

# Extract the logistic regression model from the pipeline
logistic_regression_model = model.named_steps["logisticregression"]

# Get the coefficients from the logistic regression model
coefficients = logistic_regression_model.coef_[0]

# Create a DataFrame to display feature names and their corresponding coefficients
feature_importance = pd.DataFrame({"Feature": X.columns, "Coefficient": coefficients})

# Calculate the absolute values of coefficients to determine their impact regardless of direction (positive/negative)
feature_importance["Absolute_Coefficient"] = feature_importance["Coefficient"].abs()

# Sort the features by their absolute coefficient values in descending order
feature_importance = feature_importance.sort_values(
    by="Absolute_Coefficient", ascending=False
)

# Display the sorted feature importance
feature_importance

### Exact + Fuzzy Match

In [ ]:
import pandas as pd
import numpy as np

base_match_variable = "month_year"
final_refine_variable = "cluster_mrr_total"
match_order = feature_importance['Feature'].head(6).tolist()
# Directly specifying the match_order list with predefined order
# match_order = [
#     "org_latest_mrr",
#     "instance_size",
#     "org_plan_type",
#     "self_serve_or_sales_sold",
#     "topology",
#     "cluster_age_month",
# ]

data.loc[:, "pair_index"] = pd.NA
global_pair_index = 0

unique_months = data[base_match_variable].drop_duplicates()

for month in unique_months:
    month_data = data[data[base_match_variable] == month].copy()

    # Remove outliers for 'cluster_mrr_total'
    Q1 = month_data[final_refine_variable].quantile(0.25)
    Q3 = month_data[final_refine_variable].quantile(0.75)
    IQR = Q3 - Q1
    # Keep rows that are not considered outliers
    month_data = month_data[
        (month_data[final_refine_variable] >= (Q1 - 1.5 * IQR))
        & (month_data[final_refine_variable] <= (Q3 + 1.5 * IQR))
    ]

    treatment_data = month_data[
        (month_data["group_assignment"] == "treatment group")
        & (month_data["same_month_indicator"] == 1)
    ]
    control_data = month_data[month_data["group_assignment"] == "control group"].copy()

    for t_index, t_row in treatment_data.iterrows():
        potential_matches = control_data.copy()
        match_found = False  # Flag to indicate if a match has been found

        for feature in match_order:
            initial_potential_matches = (
                potential_matches.copy()
            )  # Keep a copy before filtering

            if feature in ["org_latest_mrr", "cluster_age_month"]:  # Numerical features
                match_value = t_row[feature]
                potential_matches = potential_matches[
                    (potential_matches[feature] >= match_value * 0.9)
                    & (potential_matches[feature] <= match_value * 1.1)
                ]
            else:  # Categorical features
                potential_matches = potential_matches[
                    potential_matches[feature] == t_row[feature]
                ]

            if potential_matches.empty:
                potential_matches = initial_potential_matches  # Restore before filtering if no match found
            else:
                match_found = True  # Update flag when matches are found

            if len(potential_matches) < 3:
                break

        # If matches are found for any feature
        if match_found and not potential_matches.empty:
            potential_matches["difference"] = abs(
                potential_matches[final_refine_variable] - t_row[final_refine_variable]
            )
            closest_match_index = potential_matches["difference"].idxmin()

            # Update 'pair_index' for both treatment and matched control
            data.loc[t_index, "pair_index"] = global_pair_index
            data.loc[closest_match_index, "pair_index"] = global_pair_index
            global_pair_index += 1  # Increment for the next pair

            # Remove the matched control to avoid reusing
            control_data = control_data.drop(closest_match_index)

# Convert 'pair_index' to Int64 and filter unmatched entries
data["pair_index"] = data["pair_index"].astype("Int64")
final_matched_dataset = data.dropna(subset=["pair_index"])

### Balance Check

In [ ]:
# Calculate unique clusters in the treatment group from original data
unique_clusters_treatment_original = final_matched_dataset[
    final_matched_dataset["group_assignment"] == "treatment group"
]["cluster_id"].nunique()
print(f"Unique clusters in treatment: {unique_clusters_treatment_original}")

# Calculate unique clusters in both groups from final_matched_dataset
unique_clusters_treatment_final = final_matched_dataset[
    final_matched_dataset["group_assignment"] == "treatment group"
]["cluster_id"].nunique()
unique_clusters_control_final = final_matched_dataset[
    final_matched_dataset["group_assignment"] == "control group"
]["cluster_id"].nunique()
print(
    f"Unique clusters in treatment: {unique_clusters_treatment_final}, control: {unique_clusters_control_final}"
)

# Calculate total cluster counts in both groups
total_clusters_treatment_final = final_matched_dataset[
    final_matched_dataset["group_assignment"] == "treatment group"
]["cluster_id"].count()
total_clusters_control_final = final_matched_dataset[
    final_matched_dataset["group_assignment"] == "control group"
]["cluster_id"].count()
print(
    f"Total clusters in treatment: {total_clusters_treatment_final}, control: {total_clusters_control_final}"
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define categorical and numerical features
categorical_features = [
    "org_plan_type",
    "is_cross_region",
    "instance_size",
    "topology",
    "cluster_mdb_major_version",
    "org_country",
    "email_segment_clean",
    "marketing_channel_group",
    "account_segment",
    "self_serve_or_sales_sold",
]

numerical_features = [
    "cluster_age_month",
    "disk_size_gb",
    "org_latest_mrr",
    "reads_per_second_monthly_daily_avg",
    "writes_per_second_monthly_daily_avg",
    "connections_monthly_daily_avg",
    "opcounter_cmd_monthly_daily_avg",
    "db_data_size_monthly_daily_avg",
    "cluster_mrr_total",
]

# Filter data for Feb 2023
feb_2023_data = data[
    (data["ds"].dt.year == 2023)
    & (data["ds"].dt.month == 2)
    & (
        (data["group_assignment"] == "treatment group")
        | (data["group_assignment"] == "control group")
    )
]

# Plotting categorical features with count plots
fig, axes = plt.subplots(
    len(categorical_features),
    2,
    figsize=(15, 5 * len(categorical_features)),
    squeeze=False,
)
for i, feature in enumerate(categorical_features):
    # Before Matching - Feb 2023 Data
    sns.countplot(x=feature, hue="group_assignment", data=feb_2023_data, ax=axes[i, 0])
    axes[i, 0].set_title(f"Before Matching (Feb 2023) - {feature}")
    axes[i, 0].tick_params(axis="x", rotation=45)
    axes[i, 0].set_xlabel("")  # Remove x-label to save space

    # After Matching
    sns.countplot(
        x=feature, hue="group_assignment", data=final_matched_dataset, ax=axes[i, 1]
    )
    axes[i, 1].set_title(f"After Matching - {feature}")
    axes[i, 1].tick_params(axis="x", rotation=45)
    axes[i, 1].set_xlabel("")  # Remove x-label to save space

plt.tight_layout(h_pad=3)  # Adjust vertical spacing to prevent overlap
plt.show()

# Now, plotting numerical features with KDE plots
fig, axes = plt.subplots(
    len(numerical_features), 2, figsize=(15, 5 * len(numerical_features)), squeeze=False
)
for i, feature in enumerate(numerical_features):
    # Before Matching - Feb 2023 Data
    sns.kdeplot(
        data=feb_2023_data,
        x=feature,
        hue="group_assignment",
        ax=axes[i, 0],
        common_norm=False,
    )
    axes[i, 0].set_title(f"Before Matching (Feb 2023) - {feature}")

    # After Matching
    sns.kdeplot(
        data=final_matched_dataset,
        x=feature,
        hue="group_assignment",
        ax=axes[i, 1],
        common_norm=False,
    )
    axes[i, 1].set_title(f"After Matching - {feature}")

plt.tight_layout(h_pad=3)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Filter for '2023-02-01' in 'data' for before matching, and use 'final_matched_dataset' for after matching
before_matching_data = data[data["ds"] == "2023-02-01"][
    ["group_assignment", "cluster_mrr_total"]
].copy()
before_matching_data["Match_Status"] = "Before Matching"

after_matching_data = final_matched_dataset[
    ["group_assignment", "cluster_mrr_total"]
].copy()
after_matching_data["Match_Status"] = "After Matching"

# Combine and plot without outliers
combined_data = pd.concat([before_matching_data, after_matching_data])
plt.figure(figsize=(10, 6))
sns.boxplot(
    x="Match_Status",
    y="cluster_mrr_total",
    hue="group_assignment",
    data=combined_data,
    showfliers=False,
)
plt.title("Cluster MRR Total Before and After Matching")
plt.xlabel("Matching Status")
plt.ylabel("Cluster MRR Total")
plt.legend(title="Group Assignment")
plt.tight_layout()
plt.show()

# Analysis

### Data Manipulation

In [ ]:
import pandas as pd
from datetime import datetime

# Copy df_raw to avoid altering the original DataFrame
data = df_raw.copy()

# Ensure 'ds' columns in the copied data and final_matched_dataset are datetime objects
data["ds"] = pd.to_datetime(data["ds"])
final_matched_dataset["ds"] = pd.to_datetime(final_matched_dataset["ds"])

# Merge final_matched_dataset with the filtered copied data on 'cluster_id'
merged_data = pd.merge(
    final_matched_dataset[["cluster_id", "ds"]],
    data[["cluster_id", "ds", "cluster_consumption_revenue"]],
    on="cluster_id",
    how="left",
    suffixes=("_final", "_raw"),
)

# Filter to include records where 'ds_raw' is equal to or after 'ds_final'
filtered_data = merged_data[merged_data["ds_raw"] >= merged_data["ds_final"]].copy()

# Calculate the month difference
filtered_data["month_diff"] = (
    (filtered_data["ds_raw"].dt.year - filtered_data["ds_final"].dt.year) * 12
    + filtered_data["ds_raw"].dt.month
    - filtered_data["ds_final"].dt.month
)

# Pivot the table based on 'month_diff' for MRR totals
pivot_mrr_totals = filtered_data.pivot_table(
    index="cluster_id",
    columns="month_diff",
    values="cluster_consumption_revenue",
    aggfunc="first",
).reset_index()

# Flatten the columns and rename them to "Month X"
pivot_mrr_totals.columns = ["cluster_id"] + [
    f"Month {col}" if isinstance(col, int) else col
    for col in pivot_mrr_totals.columns[1:]
]

# Merge the pivot table back into final_matched_dataset
final_dataset_with_monthly_mrr = pd.merge(
    final_matched_dataset, pivot_mrr_totals, on="cluster_id", how="left"
)

# Fixed date for imputation
fixed_date = datetime(2024, 2, 1)
mrr_columns = [col for col in final_dataset_with_monthly_mrr.columns if "Month" in col]


# Function to impute MRR values
def impute_mrr(row):
    for col in mrr_columns:
        month_num = int(col.split(" ")[1]) if "Month" in col else 0
        month_diff = (
            (fixed_date.year - row["ds"].year) * 12 + fixed_date.month - row["ds"].month
        )
        if pd.isnull(row[col]) and month_num <= month_diff:
            row[col] = 0
    return row


# Apply the imputation row-wise
final_dataset_with_monthly_mrr = final_dataset_with_monthly_mrr.apply(
    impute_mrr, axis=1
)

# Display the result
final_dataset_with_monthly_mrr.head()

### Average MRR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sem, t, mstats

# Dynamically generate MRR columns
mrr_columns = [col for col in final_dataset_with_monthly_mrr.columns if "Month" in col]
mrr_columns = sorted(mrr_columns, key=lambda x: int(x.split(" ")[-1]))[:12]


def calculate_means_and_ci(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    std_err = sem(data) if n > 1 else 0
    margin = std_err * t.ppf((1 + confidence) / 2, n - 1)
    return mean, mean - margin, mean + margin


def winsorize_by_month(data, mrr_columns):
    for month in mrr_columns:
        # Select non-zero and non-null values for winsorization
        valid_values = data[(data[month] != 0) & (data[month].notna())][month]
        winsorized_values = mstats.winsorize(valid_values, limits=[0.05, 0.05])

        # Update the dataframe with winsorized values for non-zero and non-null entries
        data.loc[(data[month] != 0) & (data[month].notna()), month] = winsorized_values

    return data


# Apply winsorization to the dataset
final_dataset_with_monthly_mrr = winsorize_by_month(
    final_dataset_with_monthly_mrr, mrr_columns
)

original_group_colors = {"treatment group": "red", "control group": "skyblue"}
group_rename = {
    "treatment group": "Clusters w/ Product",
    "control group": "Clusters w/o Product",
}

plt.figure(figsize=(12, 6))

for original_group, color in original_group_colors.items():
    new_group_name = group_rename[original_group]
    means = []
    lower_cis = []
    upper_cis = []
    for month in mrr_columns:  # Loop through the filtered/selected month columns
        month_data = final_dataset_with_monthly_mrr[
            (final_dataset_with_monthly_mrr["group_assignment"] == original_group)
            & final_dataset_with_monthly_mrr[month].notna()
        ][month]
        mean, lower_ci, upper_ci = calculate_means_and_ci(month_data)
        means.append(mean)
        lower_cis.append(lower_ci)
        upper_cis.append(upper_ci)

    plt.plot(mrr_columns, means, marker="o", label=new_group_name, color=color)
    plt.fill_between(mrr_columns, lower_cis, upper_cis, color=color, alpha=0.2)

    for i, mean in enumerate(means):
        plt.text(
            mrr_columns[i],
            mean + (max(upper_cis) - min(lower_cis)) * 0.05,
            f"${mean:.0f}",
            ha="center",
            va="bottom",
            fontsize=8,
            color=color,
        )

plt.title("Monthly Average MRR Since Clusters First Acquired the Product")
plt.xlabel("Month")
plt.ylabel("Average MRR ($)")
plt.legend(loc="upper left")
plt.ylim(bottom=0)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sem, t

plt.figure(figsize=(12, 6))

group_means = {}

for original_group in ["treatment group", "control group"]:
    means = []
    lower_cis = []
    upper_cis = []
    for month in mrr_columns:
        month_data = final_dataset_with_monthly_mrr[
            (final_dataset_with_monthly_mrr["group_assignment"] == original_group)
            & final_dataset_with_monthly_mrr[month].notna()
        ][month]
        mean, lower_ci, upper_ci = calculate_means_and_ci(month_data)
        means.append(mean)
        lower_cis.append(lower_ci)
        upper_cis.append(upper_ci)
    group_means[original_group] = (means, lower_cis, upper_cis)

# Calculate percentage of control group mean MRR to treatment group mean MRR
treatment_means, _, _ = group_means["treatment group"]
control_means, _, _ = group_means["control group"]
percentage_means = [
    (control / treatment) * 100 if treatment else 0
    for control, treatment in zip(control_means, treatment_means)
]

# Calculate the average of the percentage means
average_percentage_mean = np.mean(percentage_means[1:])

# Plot the percentage line in green
plt.plot(
    mrr_columns,
    percentage_means,
    marker="o",
    color="green",
    label="Revenue Impact Baseline",
)

# Adding data labels for the percentage
for i, txt in enumerate(percentage_means):
    plt.annotate(
        f"{txt:.0f}%",
        (mrr_columns[i], percentage_means[i]),
        textcoords="offset points",
        xytext=(0, 10),
        ha="center",
    )

# Plot the average of the percentage means as a horizontal line
plt.axhline(
    y=average_percentage_mean, color="r", linestyle="--", label=f"Baseline Average"
)

# Add a label for the average line
# Adjust the x position as needed to place the label at the desired location on your chart
x_position_for_average_label = len(mrr_columns) - 10  # This puts it at the last month
plt.text(
    x_position_for_average_label,
    average_percentage_mean,
    f" Average (excl Month 0): {average_percentage_mean:.0f}%",
    verticalalignment="top",
    horizontalalignment="right",
    color="red",
)

plt.title("Monthly MRR of Clusters w/o Product as a Percentage of Clusters w/ Product")
plt.xlabel("Month")
plt.ylabel("Percentage")
plt.legend(loc="upper left")

# Increase the y-axis limit
plt.ylim(bottom=0, top=120)

plt.tight_layout()
plt.show()

### Sample Size

In [ ]:
# Continuing from the previous script

plt.figure(figsize=(12, 6))

for original_group, color in original_group_colors.items():
    sample_sizes = []  # List to hold the sample size for each month
    for month in mrr_columns:
        # For each month, calculate the sample size of the non-null data for the current group
        sample_size = len(
            final_dataset_with_monthly_mrr[
                (final_dataset_with_monthly_mrr["group_assignment"] == original_group)
                & final_dataset_with_monthly_mrr[month].notna()
            ][month]
        )
        sample_sizes.append(sample_size)

    plt.plot(
        mrr_columns,
        sample_sizes,
        marker="o",
        linestyle="-",
        label=group_rename[original_group],
        color=color,
    )

    for i, size in enumerate(sample_sizes):
        plt.text(
            mrr_columns[i],
            size,
            f"{size}",
            ha="center",
            va="bottom",
            fontsize=8,
            color=color,
        )

plt.title("Sample Size per Month")
plt.xlabel("Month")
plt.ylabel("Sample Size")
plt.legend(loc="upper right")
plt.ylim(bottom=0)
plt.tight_layout()
plt.show()

# Revenue Allocation

### Approach #1 - Static Allocation

In [ ]:
# # Use previous percentage means list
# percentage_means = [
#     97.804844047644,
#     92.62622004657587,
#     87.52743197833564,
#     84.75675652189551,
#     83.95310838499024,
#     81.1257192970022,
#     79.40193019770567,
#     77.0675077182838,
#     76.57380869819313,
#     76.95478784212231,
#     72.10422499840014,
#     74.39279915640566,
# ]

# # Calculate the average of the percentage means
# average_percentage_mean = np.mean(percentage_means[1:])

In [ ]:
import pandas as pd
import numpy as np

# Create a copy of the original DataFrame
data = df_raw.copy()

# Ensure both 'ds' and product_ds columns in 'data' DataFrame are in datetime format
data["ds"] = pd.to_datetime(data["ds"])
data[product_ds] = pd.to_datetime(data[product_ds])

# Filter rows based on the condition that product_ds >= 'ds' and product_ds < '2024-02-01'
filtered_data = data[
    (data[product_ds].dt.to_period("M") <= data["ds"].dt.to_period("M"))
    & (data["ds"] <= pd.Timestamp("2024-01-01"))
]

# Calculate the total sum of MRR for the filtered data
total_mrr_sum = filtered_data["cluster_consumption_revenue"].sum()

# Print the total sum of MRR formatted with no decimals
print(f"Total sum of MRR for filtered clusters: ${total_mrr_sum:,.0f}")

# Calculate and print the number of unique clusters
unique_clusters_count = filtered_data["cluster_id"].nunique()
print(f"Number of unique clusters: {unique_clusters_count}")

# Get the baseline MRR percentage from 'average_percentage_mean'
baseline_mrr_percentage = average_percentage_mean

# Print the baseline MRR percentage with no decimals
print(f"Baseline MRR Percentage (average): {baseline_mrr_percentage:.0f}%")

# Calculate the incremental revenue percentage
incremental_revenue_percentage = 100 - baseline_mrr_percentage

# Print the incremental revenue percentage with no decimals
print(
    f"Incremental Revenue Percentage (1 - Baseline): {incremental_revenue_percentage:.0f}%"
)

# Calculate incremental revenue from the usage of product
incremental_revenue = total_mrr_sum * ((1 - baseline_mrr_percentage / 100))

# Print the incremental revenue with no decimals
print(f"Incremental revenue from product usage: ${incremental_revenue:,.0f}")

### Approach #2: Dynamic Allocation

In [ ]:
from datetime import datetime

# Copy original DataFrame
data = df_raw.copy()

# Convert to datetime format and adjust to the first day of the month
data["ds"] = pd.to_datetime(data["ds"]).dt.to_period("M").dt.to_timestamp()
data[product_ds] = pd.to_datetime(data[product_ds]).dt.to_period("M").dt.to_timestamp()

# Filter data based on conditions
filtered_data = data[
    (data[product_ind] == 1)
    & (data[product_ds] <= data["ds"])  # Direct comparison using first day of the month
    & (data["ds"] >= "2023-02-01")  # Start date filter
    & (data["ds"] <= "2024-01-31")  # End date filter
].copy()

In [ ]:
from dateutil.relativedelta import relativedelta

# Calculate total month difference between 'ds' and 'product_ds' columns considering years, and cap at 11
filtered_data["month_diff"] = filtered_data.apply(
    lambda row: min(
        relativedelta(row["ds"], row[product_ds]).years * 12
        + relativedelta(row["ds"], row[product_ds]).months,
        11,
    ),
    axis=1,
)

# Group by month difference and sum total cluster_consumption_revenue
monthly_mrr = (
    filtered_data.groupby("month_diff")["cluster_consumption_revenue"]
    .sum()
    .reset_index()
)

In [ ]:
import pandas as pd
from tabulate import tabulate

# Create a copy of the DataFrame to work with
adjusted_mrr = monthly_mrr.copy()

# Add original total revenue to the copy for reference and rename the columns
adjusted_mrr["original_revenue"] = adjusted_mrr["cluster_consumption_revenue"]
adjusted_mrr.rename(
    columns={
        "month_diff": "product_tenure",
        "cluster_consumption_revenue": "attributed_consumption_revenue",
        "original_revenue": "cluster_revenue",
    },
    inplace=True,
)

# Update the code to include the percentage columns and adjusted revenue
for i, baseline_percentage in enumerate(percentage_means):
    month = i
    if month >= 0:
        # Calculate the adjusted revenue
        adjusted_factor = 1 - (baseline_percentage / 100)
        adjusted_revenue = (
            adjusted_mrr.loc[
                adjusted_mrr["product_tenure"] == month,
                "attributed_consumption_revenue",
            ]
            * adjusted_factor
        )

        # Update the DataFrame with the new adjusted revenue
        adjusted_mrr.loc[
            adjusted_mrr["product_tenure"] == month, "attributed_consumption_revenue"
        ] = adjusted_revenue

        # Store the percentage and 1-percentage with two decimal places
        adjusted_mrr.loc[
            adjusted_mrr["product_tenure"] == month, "baseline_percentage"
        ] = f"{baseline_percentage:.2f}%"
        adjusted_mrr.loc[
            adjusted_mrr["product_tenure"] == month, "attributed_revenue_percentage"
        ] = f"{adjusted_factor:.2%}"

# Calculate the sum of the adjusted and original revenues
total_adjusted_revenue = adjusted_mrr["attributed_consumption_revenue"].sum()
total_cluster_revenue = adjusted_mrr["cluster_revenue"].sum()

# Format the revenue columns as currency in the adjusted DataFrame
adjusted_mrr["cluster_revenue"] = adjusted_mrr["cluster_revenue"].apply(
    lambda x: f"${x:,.2f}"
)
adjusted_mrr["attributed_consumption_revenue"] = adjusted_mrr[
    "attributed_consumption_revenue"
].apply(lambda x: f"${x:,.2f}")

# Append total rows for both revenues
total_row = pd.DataFrame(
    {
        "product_tenure": ["Total"],
        "cluster_revenue": [f"${total_cluster_revenue:,.2f}"],
        "attributed_consumption_revenue": [f"${total_adjusted_revenue:,.2f}"],
    }
)
adjusted_mrr = pd.concat([adjusted_mrr, total_row], ignore_index=True)

# Print the adjusted DataFrame using tabulate for a nicer table format
print(tabulate(adjusted_mrr, headers="keys", tablefmt="psql"))

# Print the total revenues with formatting
print(f"\nTotal Cluster Revenue: ${total_cluster_revenue:,.2f}")
print(f"Total Attributed Consumption Revenue: ${total_adjusted_revenue:,.2f}")

# Appendix

### Multiple Product Usage

In [ ]:
# import pandas as pd
# from itertools import combinations

# # Create a copy of the original DataFrame
# data = df_raw.copy()

# # Define product columns based on your data sample
# product_columns = [
#     "text_search_ind",
#     "vector_search_ind",
#     "timeseries_ind",
#     "sync_ind",
#     "triggers_functions_ind",
#     "charts_ind",
#     "onlinearchive_ind",
#     "adf_ind",
#     "sql_ind",
# ]

# # Total unique clusters
# total_unique_clusters = data["cluster_id"].nunique()


# # Helper function to find clusters with >=N products
# def find_clusters_with_n_or_more_products(df, product_columns, n):
#     clusters = set()
#     for combo_length in range(n, len(product_columns) + 1):
#         for combo in combinations(product_columns, combo_length):
#             temp_df = df[df[list(combo)].sum(axis=1) >= n]
#             clusters.update(temp_df["cluster_id"].unique())
#     return clusters


# # Clusters with at least 1 product
# clusters_with_at_least_1_product = find_clusters_with_n_or_more_products(
#     data, product_columns, 1
# )

# # Clusters with at least 2 products
# clusters_with_at_least_2_products = find_clusters_with_n_or_more_products(
#     data, product_columns, 2
# )

# # Clusters with at least 3 products
# clusters_with_at_least_3_products = find_clusters_with_n_or_more_products(
#     data, product_columns, 3
# )

# # Print the results
# print(f"Total Unique Clusters: {total_unique_clusters}")
# print(
#     f"Unique Clusters with at Least 1 Product: {len(clusters_with_at_least_1_product)}"
# )
# print(
#     f"Unique Clusters with at Least 2 Products: {len(clusters_with_at_least_2_products)}"
# )
# print(
#     f"Unique Clusters with at Least 3 Products: {len(clusters_with_at_least_3_products)}"
# )

In [ ]:
# # Total MRR for all clusters
# data = data[data["ds"] <= "2024-01-01"].copy()
# total_mrr_all_clusters = data["cluster_consumption_revenue"].sum()

# # Calculating MRR and counting unique clusters
# mrr_1_product = data[data["cluster_id"].isin(clusters_with_at_least_1_product)][
#     "cluster_consumption_revenue"
# ].sum()
# count_1_product = len(clusters_with_at_least_1_product)

# mrr_2_products = data[data["cluster_id"].isin(clusters_with_at_least_2_products)][
#     "cluster_consumption_revenue"
# ].sum()
# count_2_products = len(clusters_with_at_least_2_products)

# mrr_3_products = data[data["cluster_id"].isin(clusters_with_at_least_3_products)][
#     "cluster_consumption_revenue"
# ].sum()
# count_3_products = len(clusters_with_at_least_3_products)

# # Calculating percentages
# percentage_mrr_1_product = (mrr_1_product / total_mrr_all_clusters) * 100
# percentage_mrr_2_products = (mrr_2_products / total_mrr_all_clusters) * 100
# percentage_mrr_3_products = (mrr_3_products / total_mrr_all_clusters) * 100

# # Print the results with unique cluster counts
# print(f"Total MRR for all clusters: ${total_mrr_all_clusters:,.0f}")
# print(
#     f"Clusters with at least 1 product: {count_1_product}, Total MRR: ${mrr_1_product:,.0f} ({percentage_mrr_1_product:.2f}%)"
# )
# print(
#     f"Clusters with at least 2 products: {count_2_products}, Total MRR: ${mrr_2_products:,.0f} ({percentage_mrr_2_products:.2f}%)"
# )
# print(
#     f"Clusters with at least 3 products: {count_3_products}, Total MRR: ${mrr_3_products:,.0f} ({percentage_mrr_3_products:.2f}%)"
# )

### Churn Chart

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'final_dataset_with_monthly_mrr' is your DataFrame and is already loaded

# Iterate over the DataFrame to create churn indicator columns based on the specified conditions
for month in range(13):
    mrr_col = f'Month {month}'  # Corrected column names
    churn_col = f'Month {month}_churn'
    
    # Define conditions for churn indicator
    conditions = [
        (final_dataset_with_monthly_mrr[mrr_col] == 0),  # Churn if value is 0
        (final_dataset_with_monthly_mrr[mrr_col].notnull())  # Not churn if value is not 0 and not null
    ]
    choices = [1, 0]  # Churn indicator values
    final_dataset_with_monthly_mrr[churn_col] = np.select(conditions, choices, default=np.nan)

# Initialize an empty DataFrame to store churn rates
churn_rates = pd.DataFrame()

# List of churn indicator columns
churn_indicator_columns = [f'Month {i}_churn' for i in range(13)]

# Calculate churn rates
for month_idx, col in enumerate(churn_indicator_columns):
    # Group by 'group_assignment' and calculate churned clusters and total clusters with data
    churn_data = final_dataset_with_monthly_mrr.groupby('group_assignment')[col].agg(
        churned_clusters=np.sum,
        total_clusters_with_data=pd.Series.count
    ).reset_index()

    # Calculate churn rate
    churn_data['churn_rate'] = (churn_data['churned_clusters'] / churn_data['total_clusters_with_data']) * 100
    churn_data['month'] = f'Month {month_idx}'

    # Append to the churn_rates DataFrame
    churn_rates = pd.concat([churn_rates, churn_data], ignore_index=True)

# Filter to include only Month 0 to Month 11
churn_rates = churn_rates[churn_rates['month'].isin([f'Month {i}' for i in range(12)])]

# Set seaborn style for the plot
sns.set(style="white", rc={"axes.grid": False})

# Create a line plot
plt.figure(figsize=(10, 6))
lineplot = sns.lineplot(data=churn_rates, x='month', y='churn_rate', hue='group_assignment', marker='o')

# Plot enhancements
plt.title('Monthly Churn Rate for Control & Treatment Groups', fontsize=15)
plt.xlabel('Month', fontsize=15)
plt.ylabel('Churn Rate (%)', fontsize=15)
plt.xticks(rotation=45)
plt.legend(title='Group Assignment', title_fontsize='10', fontsize='11')

# Adding data labels
for index, row in churn_rates.iterrows():
    plt.text(x=row['month'], y=row['churn_rate']+0.5,  # Slightly adjust the height for visibility
             s=f'{row["churn_rate"]:.0f}%', ha='center', color='black', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# data = df_raw.copy()
# data["ds"] = pd.to_datetime(data["ds"])

# # Reference date
# reference_date = pd.to_datetime("2023-02-01")

# # Extract year and month for both `ds` and `reference_date`, then calculate the month difference
# data["year"], data["month"] = data["ds"].dt.year, data["ds"].dt.month
# ref_year, ref_month = reference_date.year, reference_date.month

# # Calculate month difference as 12 times the year difference plus the month difference
# data["month_diff"] = (data["year"] - ref_year) * 12 + (data["month"] - ref_month)

# # Now, you can pivot your table with this adjusted 'month_diff'
# pivot_table = data.pivot_table(
#     index="cluster_id",
#     columns="month_diff",
#     values="cluster_consumption_revenue",
#     fill_value=0,
#     aggfunc="sum",
# ).reset_index()

# # Clean-up: You might want to drop the temporary columns 'year' and 'month' if they are no longer needed
# data.drop(columns=["year", "month"], inplace=True)

# # Filter rows where month 0 is not zero
# pivot_table = pivot_table[pivot_table[0] != 0]

# # Calculate churn rates for each month from 0 to 11
# churn_rates = []
# for month in range(12):  # Iterate through months 0 to 11
#     churn_count = pivot_table[pivot_table[month] == 0].shape[0]
#     total_count = pivot_table.shape[0]
#     churn_rate = churn_count / total_count
#     churn_rates.append(churn_rate)

# # Plot churn rates
# plt.figure(figsize=(14, 8))  # Set figure size
# plt.plot(range(12), churn_rates, marker="o", linestyle="-", color="b")

# # Adding data labels for each point
# for i, rate in enumerate(churn_rates):
#     plt.text(i, rate, f"{rate:.0%}", ha="center", va="bottom")

# plt.title("Churn Rate for 2023-02 Active Cluster Cohort", fontsize=15)
# plt.xlabel("Month")
# plt.ylabel("Churn Rate")
# plt.xticks(range(12), labels=[f"Month {i}" for i in range(12)])
# plt.grid(False)
# plt.show()

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# data = df_raw.copy()

# data["ds"] = pd.to_datetime(data["ds"])
# data["created_at"] = pd.to_datetime(
#     data["created_at"]
# )  # Ensure 'created_at' is in datetime format

# # Narrow down to the population created in February 2023
# start_date = pd.to_datetime("2023-02-01")
# end_date = pd.to_datetime("2023-02-28")
# data = data[(data["created_at"] >= start_date) & (data["created_at"] <= end_date)]

# # Reference date
# reference_date = pd.to_datetime("2023-02-01")

# # Extract year and month for both `ds` and `reference_date`, then calculate the month difference
# data["year"], data["month"] = data["ds"].dt.year, data["ds"].dt.month
# ref_year, ref_month = reference_date.year, reference_date.month

# # Calculate month difference as 12 times the year difference plus the month difference
# data["month_diff"] = (data["year"] - ref_year) * 12 + (data["month"] - ref_month)

# # Now, you can pivot your table with this adjusted 'month_diff'
# pivot_table = data.pivot_table(
#     index="cluster_id",
#     columns="month_diff",
#     values="cluster_consumption_revenue",
#     fill_value=0,
#     aggfunc="sum",
# ).reset_index()

# # Clean-up: You might want to drop the temporary columns 'year' and 'month' if they are no longer needed
# data.drop(columns=["year", "month"], inplace=True)

# # Filter rows where month 0 is not zero
# pivot_table = pivot_table[pivot_table[0] != 0]

# # Calculate churn rates for each month from 0 to 11
# churn_rates = []
# for month in range(12):  # Iterate through months 0 to 11
#     churn_count = pivot_table[pivot_table[month] == 0].shape[0]
#     total_count = pivot_table.shape[0]
#     churn_rate = churn_count / total_count
#     churn_rates.append(churn_rate)

# # Plot churn rates
# plt.figure(figsize=(14, 8))  # Set figure size
# plt.plot(range(12), churn_rates, marker="o", linestyle="-", color="b")

# # Adding data labels for each point
# for i, rate in enumerate(churn_rates):
#     plt.text(i, rate, f"{rate:.0%}", ha="center", va="bottom")

# plt.title("Churn Rate for Feb 2023 Created Cluster Cohort", fontsize=15)
# plt.xlabel("Month")
# plt.ylabel("Churn Rate")
# plt.xticks(range(12), labels=[f"Month {i}" for i in range(12)])
# plt.grid(False)
# plt.show()

### Small Sample Size

In [ ]:
# import matplotlib.pyplot as plt

# # Data setup
# products = ["text_search", "vector_search", "time_series", "sync", "trigger and functions",
#             "charts", "online archive", "adf", "sql"]
# unique_clusters = [5443, 365, 1424, 1109, 6097, 7931, 1976, 2194, 186]
# definitions = ["qualification", "qualification", "qualification", "qualification",
#                "activation", "activation", "activation", "activation", "activation"]

# # Choose colors based on 'definitions'
# colors = ['#1f77b4' if defi == "qualification" else '#ff7f0e' for defi in definitions]

# # Create bar chart
# fig, ax = plt.subplots(figsize=(10, 6))
# bars = ax.barh(products, unique_clusters, color=colors)

# # Adding grid, background color, and customizing axes
# ax.set_facecolor('#f9f9f9')
# ax.grid(True, which='both', linestyle='--', linewidth=0.5)
# ax.set_axisbelow(True)

# # Annotating small sample sizes
# for bar, value in zip(bars, unique_clusters):
#     if value < 1000:
#         ax.annotate(f'Small sample: {value}', (value + 100, bar.get_y() + bar.get_height()/2),
#                     textcoords="offset points", xytext=(5,0), va='center', fontsize=8, color='red')

# # Enhance labels and title
# ax.set_xlabel('Number of Unique Clusters')
# ax.set_title('Product Unique Clusters Overview', fontsize=14, fontweight='bold')
# ax.xaxis.set_tick_params(labelsize=10)
# ax.yaxis.set_tick_params(labelsize=10)

# # Add legend with a title
# qualification_bar = plt.Rectangle((0,0),1,1, fc="#1f77b4", edgecolor='none')
# activation_bar = plt.Rectangle((0,0),1,1, fc='#ff7f0e', edgecolor='none')
# ax.legend([qualification_bar, activation_bar], ['Qualification', 'Activation'], loc='upper right', title="Definition Used")

# # Add concise note about the data context and adjust position
# note_text = ("Note: The numbers shown are the total clusters available for revenue distribution per product during FY24")
# plt.gcf().subplots_adjust(bottom=0.15)  # Adjust the bottom to prevent overlap
# plt.figtext(0.5, 0.01, note_text, wrap=True, horizontalalignment='center', fontsize=10, color='darkred')

# # Show the plot
# plt.show()
